# Load audios

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
import os
import shutil
import glob
import pandas as pd
import numpy as np
from tqdm import tqdm

def unzip_files(zip_file_path, destination_folder):

  # destination_folder = zip_file_path.split('.zip')[0]

  # Create the destination folder if it doesn't exist
  if not os.path.exists(destination_folder):
      os.makedirs(destination_folder)

  # Open the zip file
  with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
      # Get total number of files to extract
      total_files = len(zip_ref.namelist())

      # Extract all contents to the destination folder with progress bar
      for file in tqdm(zip_ref.namelist(), total=total_files, desc=f'Extracting {zip_file_path}', unit='files'):
          zip_ref.extract(file, destination_folder)

In [ ]:
# shutil.rmtree('OriginalAudios')
# shutil.rmtree('ClonedAudiosout')
unzip_files('/content/drive/MyDrive/Colab Notebooks/TFM/Exp2/OriginalAudios.zip', 'OriginalAudios')
unzip_files('/content/drive/MyDrive/Colab Notebooks/TFM/Exp2/ClonedAudiosout.zip', 'ClonedAudiosout')

Extracting /content/drive/MyDrive/Colab Notebooks/TFM/Exp2/OriginalAudios.zip: 100%|██████████| 7171/7171 [00:31<00:00, 225.49files/s]
Extracting /content/drive/MyDrive/Colab Notebooks/TFM/Exp2/ClonedAudiosout.zip: 100%|██████████| 13355/13355 [01:35<00:00, 140.03files/s]


# Install packages

In [ ]:
!pip install jiwer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 19.1 MB/s eta 0:00:00


In [ ]:
!pip install -U openai-whisper

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 11.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using c

# Obtain transcriptions

In [ ]:
import whisper
from IPython.display import clear_output

whisper = whisper.load_model("base")

def speech_to_text(metadata_csv_path):
  metadata_df = pd.read_csv(metadata_csv_path)

  for index, row in metadata_df.iterrows():

    clear_output(wait=True)

    percentage = round(100 * (1+index) / len(metadata_df), 3)
    print(f'{percentage} %')

    asr_text = whisper.transcribe(row['Audio path'])['text']

    metadata_df.loc[index, 'Transcription'] = asr_text

  return metadata_df

100%|███████████████████████████████████████| 139M/139M [00:06<00:00, 23.8MiB/s]


In [ ]:
metadata_df = speech_to_text('metadata_with_secs.csv')

100.0 %


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


In [ ]:
# metadata_df

# Obtain WER

In [ ]:
import jiwer

transforms = jiwer.Compose(
    [
        jiwer.ExpandCommonEnglishContractions(),
        jiwer.RemoveEmptyStrings(),
        jiwer.ToLowerCase(),
        jiwer.RemoveMultipleSpaces(),
        jiwer.Strip(),
        jiwer.RemovePunctuation(),
        jiwer.ReduceToListOfListOfWords(),
    ]
)

In [ ]:
for index, row in metadata_df.iterrows():

  clear_output(wait=True)

  percentage = round(100 * (1+index) / len(metadata_df), 2)
  print(f'{percentage} %')

  reference = metadata_df.loc[index, 'Text']
  hypothesis = metadata_df.loc[index, 'Transcription']

  try:
    metadata_df.loc[index, 'WER'] = jiwer.wer(
                      reference,
                      hypothesis,
                      truth_transform=transforms,
                      hypothesis_transform=transforms,
                  )

  except:
    metadata_df.loc[index, 'WER'] = np.nan

100.0 %


In [ ]:
# metadata_df

In [ ]:
columns = metadata_df.columns.to_list()[:11] + metadata_df.columns.to_list()[-2:] + metadata_df.columns.to_list()[11:-2]
metadata_df = metadata_df[columns]
# metadata_df

In [ ]:
metadata_df.to_csv('metadata_with_wer.csv', index=False)

In [ ]:
from google.colab import files
files.download('metadata_with_wer.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>